In [76]:
import pandas as pd
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, value

# Load datasets
warehouse_capabilities = pd.read_csv(
    "data/Supply chain logistics problem - WhCapabilities.csv"
)
freight_rates = pd.read_csv("data/Supply chain logistics problem - FreightRates.csv")
plant_ports = pd.read_csv("data/Supply chain logistics problem - PlantPorts.csv")
products_per_plant = pd.read_csv(
    "data/Supply chain logistics problem - ProductsPerPlant.csv"
)
order_list = pd.read_csv("data/Supply chain logistics problem - OrderList.csv")
vmi_customers = pd.read_csv("data/Supply chain logistics problem - VmiCustomers.csv")
wh_costs = pd.read_csv("data/Supply chain logistics problem - WhCosts.csv")

In [79]:
import pandas as pd
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, value, LpStatus

# Load datasets
freight_rates = pd.read_excel("data/Supply chain logistics problem.xlsx", sheet_name="FreightRates")
warehouse_capabilities = pd.read_excel("data/Supply chain logistics problem.xlsx", sheet_name="WhCapacities")
order_list = pd.read_excel("data/Supply chain logistics problem.xlsx", sheet_name="OrderList")
plant_ports = pd.read_excel("data/Supply chain logistics problem.xlsx", sheet_name="PlantPorts")

# Convert 'rate' column to numeric by removing '$' and converting to float
freight_rates['minimum cost'] = freight_rates['minimum cost'].replace('[\$,]', '', regex=True).astype(float)

# Aggregate demand by destination port
demand_by_port = order_list.groupby('Destination Port')['Unit quantity'].sum().to_dict()
total_demand = sum(demand_by_port.values())

# Extract plant capacities
plant_capacities = warehouse_capabilities.set_index("Plant ID")['Daily Capacity '].to_dict()
total_supply = sum(plant_capacities.values())  # Calculate total supply from plant capacities

# Map plants to their respective ports
plant_to_port = plant_ports.set_index("Plant Code")['Port'].to_dict()

# Filter relevant freight rates based on available plant-port combinations
relevant_freight_rates = freight_rates[
    freight_rates['orig_port_cd'].isin(plant_to_port.values())
    & freight_rates['dest_port_cd'].isin(demand_by_port.keys())
]

# Create cost dictionary for relevant (origin, destination) pairs
freight_costs = (
    relevant_freight_rates.groupby(["orig_port_cd", "dest_port_cd"])['minimum cost']
    .min()
    .to_dict()
)

# Ensure valid pairs exist
valid_pairs = [
    (plant, "PORT09")
    for plant, port in plant_to_port.items()
    if (port, "PORT09") in freight_costs
]

# Initialize model
model = LpProblem("Supply_Chain_Optimization_Simple", LpMinimize)

# Define decision variables for shipments (for all plants)
ship_vars = {
    (plant, "PORT09"): LpVariable(f"Ship_{plant}_PORT09", lowBound=0, cat='Continuous')
    for plant in plant_capacities.keys()
}

# Objective function: Minimize transport cost
model += lpSum(
    ship_vars[(plant, "PORT09")] * freight_costs.get((plant_to_port[plant], "PORT09"), 1e6)
    for plant in plant_capacities.keys()
), "Total_Transport_Cost"

# Supply constraints: Ensure total shipments from each plant do not exceed capacity
for plant in plant_capacities.keys():
    model += ship_vars[(plant, "PORT09")] <= plant_capacities[plant], f"Supply_Constraint_{plant}"

# Demand constraint: Ensure total shipments meet total available supply
model += lpSum(
    ship_vars[(plant, "PORT09")] for plant in plant_capacities.keys()
) == total_supply, "Adjusted_Demand_Constraint"


# Solve the model
model.solve()

# Print results
print("Status:", LpStatus[model.status])
print("Total Cost:", value(model.objective))

# Print shipment decisions
for (plant, destination), var in ship_vars.items():
    if var.varValue > 0:
        print(f"{plant} -> {destination}: {var.varValue}")


Status: Optimal
Total Cost: 23146.711600000002
PLANT15 -> PORT09: 11.0
PLANT17 -> PORT09: 8.0
PLANT18 -> PORT09: 111.0
PLANT05 -> PORT09: 385.0
PLANT02 -> PORT09: 138.0
PLANT01 -> PORT09: 1070.0
PLANT06 -> PORT09: 49.0
PLANT10 -> PORT09: 118.0
PLANT07 -> PORT09: 265.0
PLANT14 -> PORT09: 549.0
PLANT16 -> PORT09: 457.0
PLANT12 -> PORT09: 209.0
PLANT11 -> PORT09: 332.0
PLANT09 -> PORT09: 11.0
PLANT03 -> PORT09: 1013.0
PLANT13 -> PORT09: 490.0
PLANT19 -> PORT09: 7.0
PLANT08 -> PORT09: 14.0
PLANT04 -> PORT09: 554.0


In [2]:
import pandas as pd

freight_rates = pd.read_excel("data/Supply chain logistics problem.xlsx", sheet_name="FreightRates")
plant_ports = pd.read_excel("data/Supply chain logistics problem.xlsx", sheet_name="PlantPorts")

# Convert 'minimum cost' column to numeric
freight_rates['minimum cost'] = freight_rates['minimum cost'].replace('[\$,]', '', regex=True).astype(float)

# Map plants to their respective ports
plant_to_port = plant_ports.set_index("Plant Code")['Port'].to_dict()

# Filter relevant freight rates
relevant_freight_rates = freight_rates[
    freight_rates["orig_port_cd"].isin(plant_to_port.values())
    & (freight_rates["dest_port_cd"] == "PORT09")
]

# Create a dictionary of freight costs for each plant to PORT09
freight_costs = relevant_freight_rates.groupby("orig_port_cd")["minimum cost"].min().to_dict()

# Print the freight costs
print("Freight Costs (Plant to PORT09):")
for plant, cost in freight_costs.items():
    print(f"{plant}: {cost}")


Freight Costs (Plant to PORT09):
PORT02: 1.43
PORT03: 19.3644
PORT04: 1.202
PORT05: 1.5584
PORT06: 1.3352
PORT07: 9.6272
PORT08: 19.0272
PORT09: 17.6888
PORT10: 1.4024
PORT11: 7.8272


In [3]:
import pandas as pd

# Load datasets
freight_rates = pd.read_excel("data/Supply chain logistics problem.xlsx", sheet_name="FreightRates")
plant_ports = pd.read_excel("data/Supply chain logistics problem.xlsx", sheet_name="PlantPorts")
warehouse_capacities = pd.read_excel("data/Supply chain logistics problem.xlsx", sheet_name="WhCapacities")

# Convert 'minimum cost' column to numeric
freight_rates['minimum cost'] = freight_rates['minimum cost'].replace('[\$,]', '', regex=True).astype(float)

# Map plants to their respective ports
plant_to_port = plant_ports.set_index("Plant Code")['Port'].to_dict()

# Create a DataFrame for capacities
capacities_df = warehouse_capacities.set_index("Plant ID")["Daily Capacity "]

# Filter relevant freight rates where destination port is PORT09
relevant_freight_rates = freight_rates[
    (freight_rates["dest_port_cd"] == "PORT09")
    & (freight_rates["orig_port_cd"].isin(plant_to_port.values()))
]

# Create a dictionary of freight costs for each plant
freight_costs = {
    plant: relevant_freight_rates.loc[
        relevant_freight_rates["orig_port_cd"] == port, "minimum cost"
    ].min()
    for plant, port in plant_to_port.items()
}

# Print plant-to-port mapping with capacities and freight costs
print("Plant-to-Port Mapping with Capacities and Freight Costs:")
for plant in plant_to_port.keys():
    print(f"{plant}: Port = {plant_to_port[plant]}, Capacity = {capacities_df[plant]}, Cost = {freight_costs[plant]}")


Plant-to-Port Mapping with Capacities and Freight Costs:
PLANT01: Port = PORT02, Capacity = 1070, Cost = 1.43
PLANT02: Port = PORT03, Capacity = 138, Cost = 19.3644
PLANT03: Port = PORT04, Capacity = 1013, Cost = 1.202
PLANT04: Port = PORT05, Capacity = 554, Cost = 1.5584
PLANT05: Port = PORT06, Capacity = 385, Cost = 1.3352
PLANT06: Port = PORT06, Capacity = 49, Cost = 1.3352
PLANT07: Port = PORT02, Capacity = 265, Cost = 1.43
PLANT08: Port = PORT04, Capacity = 14, Cost = 1.202
PLANT09: Port = PORT04, Capacity = 11, Cost = 1.202
PLANT10: Port = PORT02, Capacity = 118, Cost = 1.43
PLANT11: Port = PORT04, Capacity = 332, Cost = 1.202
PLANT12: Port = PORT04, Capacity = 209, Cost = 1.202
PLANT13: Port = PORT04, Capacity = 490, Cost = 1.202
PLANT14: Port = PORT07, Capacity = 549, Cost = 9.6272
PLANT15: Port = PORT08, Capacity = 11, Cost = 19.0272
PLANT16: Port = PORT09, Capacity = 457, Cost = 17.6888
PLANT17: Port = PORT10, Capacity = 8, Cost = 1.4024
PLANT18: Port = PORT11, Capacity = 111

In [3]:
# Check the column names in the freight_rates dataset
print("Column Names in Freight Rates Dataset:", freight_rates.columns)

# Display the first few rows for reference
print("Sample Data in Freight Rates Dataset:")
print(freight_rates.head())


Column Names in Freight Rates Dataset: Index(['Carrier', 'orig_port_cd', 'dest_port_cd', 'minm_wgh_qty',
       'max_wgh_qty', 'svc_cd', 'minimum cost', 'rate', 'mode_dsc',
       'tpt_day_cnt', 'Carrier type'],
      dtype='object')
Sample Data in Freight Rates Dataset:
  Carrier orig_port_cd dest_port_cd  minm_wgh_qty max_wgh_qty svc_cd  \
0  V444_6       PORT08       PORT09         250.0      499.99    DTD   
1  V444_6       PORT08       PORT09          65.0       69.99    DTD   
2  V444_6       PORT08       PORT09          60.0       64.99    DTD   
3  V444_6       PORT08       PORT09          50.0       54.99    DTD   
4  V444_6       PORT08       PORT09          35.0       39.99    DTD   

  minimum cost     rate mode_dsc  tpt_day_cnt Carrier type  
0      $43.23    $0.71    AIR               2  V88888888_0  
1      $43.23    $0.75    AIR               2  V88888888_0  
2      $43.23    $0.79    AIR               2  V88888888_0  
3      $43.23    $0.83    AIR               2  V888

In [5]:
print("Column Names in Warehouse Capabilities Dataset:", warehouse_capabilities.columns)
print("Sample Data in Warehouse Capabilities Dataset:")
print(warehouse_capabilities.head())


Column Names in Warehouse Capabilities Dataset: Index(['Plant ID', 'Daily Capacity '], dtype='object')
Sample Data in Warehouse Capabilities Dataset:
  Plant ID  Daily Capacity 
0  PLANT15               11
1  PLANT17                8
2  PLANT18              111
3  PLANT05              385
4  PLANT02              138


In [9]:
print(freight_rates[['orig_port_cd', 'dest_port_cd', 'rate']].head())


  orig_port_cd dest_port_cd  rate
0       PORT08       PORT09  0.71
1       PORT08       PORT09  0.75
2       PORT08       PORT09  0.79
3       PORT08       PORT09  0.83
4       PORT08       PORT09  1.06


In [10]:
demand_summary = order_list.groupby('Destination Port')['Unit quantity'].sum()
print(demand_summary)


Destination Port
PORT09    29513315
Name: Unit quantity, dtype: int64


In [11]:
for origin, destination in ship_vars:
    if origin == destination:
        model += ship_vars[(origin, destination)] == 0, f"No_Self_Loop_{origin}_{destination}"


In [12]:
for constraint in model.constraints.values():
    print(constraint)


0 <= 11
0 <= 8
0 <= 111
0 <= 385
0 <= 138
0 <= 1070
0 <= 49
0 <= 118
0 <= 265
0 <= 549
0 <= 457
0 <= 209
0 <= 332
0 <= 11
0 <= 1013
0 <= 490
0 <= 7
0 <= 14
0 <= 554
Ship_PORT02_PORT09 + Ship_PORT03_PORT09 + Ship_PORT04_PORT09 + Ship_PORT05_PORT09 + Ship_PORT06_PORT09 + Ship_PORT07_PORT09 + Ship_PORT08_PORT09 + Ship_PORT09_PORT09 + Ship_PORT10_PORT09 + Ship_PORT11_PORT09 >= 29513315
Ship_PORT09_PORT09 = 0


In [14]:
print(order_list[['Destination Port', 'Unit quantity']].sort_values(by='Unit quantity', ascending=False).head())


     Destination Port  Unit quantity
6858           PORT09         561847
6880           PORT09         469247
6883           PORT09         454079
6882           PORT09         436703
6865           PORT09         384637


In [16]:
print(warehouse_capabilities)


   Plant ID  Daily Capacity 
0   PLANT15               11
1   PLANT17                8
2   PLANT18              111
3   PLANT05              385
4   PLANT02              138
5   PLANT01             1070
6   PLANT06               49
7   PLANT10              118
8   PLANT07              265
9   PLANT14              549
10  PLANT16              457
11  PLANT12              209
12  PLANT11              332
13  PLANT09               11
14  PLANT03             1013
15  PLANT13              490
16  PLANT19                7
17  PLANT08               14
18  PLANT04              554


In [17]:
print(freight_rates[['orig_port_cd', 'dest_port_cd', 'rate']].describe())


              rate
count  1540.000000
mean      2.892987
std       4.603847
min       0.030000
25%       0.472500
50%       1.665000
75%       3.932500
max     128.030000


In [18]:
total_demand = order_list['Unit quantity'].sum()
total_supply = warehouse_capabilities['Daily Capacity '].sum()
print(f"Total Demand: {total_demand}, Total Supply: {total_supply}")


Total Demand: 29513315, Total Supply: 5791
